In [ ]:
import os 
import sys
import numpy as np
import argparse
from scipy.spatial.transform import Rotation as R 
# add catkin_ws context 
sys.path.append("/home/junting/franka_ws/devel/lib/python3.9/site-packages")

from src.lmp import *
from src.env.true_grounding_env import TrueGroundingEnv
from src.config import cfg_tabletop
import rospy 
import rospkg
import jupyros as jr

from std_msgs.msg import String, Header
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion


In [ ]:
########################################
# initialize environment
########################################
rospy.init_node('eval_code', log_level=rospy.DEBUG)
# get package root path 
pkg_root = rospkg.RosPack().get_path('instruct_to_policy')

# setup environment
env = TrueGroundingEnv(cfg_tabletop)
# env.reset()


In [ ]:
env.parse_adaptive_shape_grasp_pose("Room_Essentials_Mug_White_Yellow")

[DEBUG] [1700071201.624778, 530.156000]: Unable to connect subscriber to publisher [http://junting-shlab:33971/] for topic [/tf]: [Errno 111] Connection refused
[DEBUG] [1700071201.645319, 530.156000]: connecting to junting-shlab 53899
[DEBUG] [1700071201.706818, 530.156000]: connecting to junting-shlab 53899
[DEBUG] [1700071201.721705, 530.156000]: Unable to connect subscriber to publisher [http://junting-shlab:33971/] for topic [/tf]: [Errno 111] Connection refused
[DEBUG] [1700071201.759881, 530.156000]: connecting to junting-shlab 44845
[DEBUG] [1700071201.778225, 530.156000]: connecting to junting-shlab 44845
[DEBUG] [1700071201.930032, 530.156000]: connecting to junting-shlab 40915
[DEBUG] [1700071202.415616, 530.156000]: connecting to junting-shlab 52125
[DEBUG] [1700071202.417491, 530.156000]: connecting to junting-shlab 57389
[DEBUG] [1700071202.460689, 530.156000]: connecting to junting-shlab 52125
[DEBUG] [1700071202.552384, 530.156000]: connecting to junting-shlab 57389
[DE

[WARN] [1700071207.030833, 0.021000]: Detected jump back in time of 530.135000s. Clearing TF buffer.
[ WARN] [1700071207.030590262, 0.020000000]: Detected jump back in time of 530.136s. Clearing TF buffer.


[DEBUG] [1700071207.332668, 0.320000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.342858, 0.330000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.358680, 0.344000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.361791, 0.347000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.382961, 0.352000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.395546, 0.379000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.397242, 0.381000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.401524, 0.385000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.401873, 0.385000]: connecting to junting-shlab 60427
[DEBUG] [1700071207.433110, 0.417000]: connecting to junting-shlab 60427
[DEBUG] [1700072198.346990, 711.725000]: [/tf_static] failed to receive incoming message : unable to receive data from sender, check sender's logs for details
[DEBUG] [1700072198.350464, 711.725000]: [/tf] failed to receive incoming message : unable to receive data from

[WARN] [1700072224.529407, 0.021000]: Detected jump back in time of 711.704000s. Clearing TF buffer.
[ WARN] [1700072224.528787311, 0.021000000]: Detected jump back in time of 711.704s. Clearing TF buffer.


[DEBUG] [1700072224.803418, 0.293000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.805632, 0.297000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.809738, 0.301000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.821519, 0.311000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.823331, 0.313000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.825468, 0.315000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.845429, 0.326000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.859226, 0.345000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.860534, 0.346000]: connecting to junting-shlab 42469
[DEBUG] [1700072224.905354, 0.388000]: connecting to junting-shlab 42469
[DEBUG] [1700072937.627805, 148.260000]: [/camera_right/aligned_depth/image_raw] failed to receive incoming message : unable to receive data from sender, check sender's logs for details
[DEBUG] [1700072937.630179, 148.260000]: [/camera_right/aligned_depth/camera_info] f

In [ ]:
# Load files & data 
def filter_drawer0(processed_data, raw_data, mask):
    """
    Filter tasks involving interacting with drawer 0, since it is too far from the robot.
    """

    for i, data in enumerate(processed_data):
        if mask[i] == 0: # already filtered in previous filters 
            continue
        full_query = data['query']
        instruction = full_query.split(';')[-1]
        if 'drawer0' in instruction:
            mask[i] = 0
            
    return mask 


def filter_tasks(processed_data, raw_data):
    """
    Filter the tasks that are not suitable for the environment based on hand-crafted rules.
    """
    filter_funcs = [filter_drawer0]
    mask = np.ones((len(processed_data)))
    for func in filter_funcs:
        mask = func(processed_data, raw_data, mask)
        
    filtered_processed_data = [processed_data[i] for i in range(len(processed_data)) if mask[i] == 1]
    filtered_raw_data = [raw_data[i] for i in range(len(raw_data)) if mask[i] == 1]
    return filtered_processed_data, filtered_raw_data

processed_file_path = os.path.join(pkg_root, processed_file)
raw_file_path = os.path.join(pkg_root, raw_file)

with open(processed_file_path, 'r') as f:
    processed_data = json.load(f)
with open(raw_file_path, 'r') as f:
    raw_data = json.load(f)

# filter tasks that are not suitable for the environment
filtered_processed_data, filtered_raw_data = filter_tasks(processed_data, raw_data)

## Pick and Place

In [ ]:
env.reset()

In [ ]:
pose_msg = env.parse_grasp_pose("apple")

In [ ]:
rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)

In [ ]:
env.open_gripper()

In [ ]:
env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)

In [ ]:
env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)

In [ ]:
env.close_gripper(width=0.05, force=30)

In [ ]:
env.attach_object("apple")

In [ ]:
place_pose = env.parse_place_pose(object_name="apple", receptacle_name="white_ceramic_plate")

In [ ]:
env.move_to_pose(place_pose)
env.open_gripper()

In [ ]:
env.detach_object("apple")

## Open Drawer

In [ ]:
jr.publish('/rviz/moveit/move_marker/goal_panda_hand_tcp', PoseStamped)

In [ ]:
env.reset()
# env.open_gripper()

In [ ]:
# env.get_obj_name_list()
# [bbox.object_id for bbox in env.gazebo_gt_bboxes]
env.get_3d_bbox("cabinet.handle_0")

In [ ]:
grasp_pose = env.parse_grasp_pose(object_name="cabinet.handle_0")
print(grasp_pose)

In [ ]:
env.publish_goal_to_marker(grasp_pose)

In [ ]:
env.grasp(grasp_pose)
# env.move_to_pose(grasp_pose)

In [ ]:
env.close_gripper()

In [ ]:
# generate a horizontal trajectory to open the drawer
grasp_position = np.array([grasp_pose.position.x, grasp_pose.position.y, grasp_pose.position.z])
pull_position = grasp_position + np.array([0.2, 0, 0]).astype(float)
pull_pose = Pose(position=Point(*pull_position), orientation=grasp_pose.orientation)

In [ ]:
env.move_to_pose(pull_pose)

In [ ]:
env.open_gripper()

In [ ]:
pose_msg = env.parse_grasp_pose(object_name="apple")

rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)
 
env.open_gripper()
 
# env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)
# env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)
env.close_gripper(width=0.05, force=30)
env.attach_object("apple")


In [ ]:
env.reset()